In [1]:
# import the core map making code
from makemap import make_map_smurf, make_map_vna
# custom packages used in this example
from read_iv import read_psat
from simple_csv import read_csv
from channel_assignment import OperateTuneData
from layout_data import get_layout_data
# get a sample configuration to use with this example
from detmap_conifg_example import path_north_side_vna, path_south_side_vna, cold_ramp_file, \
    north_is_highband, vna_shift_mhz, tunefile, dark_bias_lines, design_file, mux_pos_num_to_mux_band_num_path, \
    waferfile, output_filename, output_filename_vna, tune_data_vna_output_filename

The Sample Data used for the default data not found, doing a one time download of the sample data data.
  Beginning file download of at https://drive.google.com/file/d/1G8eiJ85zVKu53GCzeVdjHEV8cWqrg6eH/view?usp=sharing
  This is ~500 Mb file so it make take a while depending on your connection speed...
Download Complete.

Extracting the zipfile...
  The zipfile extracted and unpacked.
  A cleanup was done remove original zipfile



In [2]:
# Get Metadata
# get the design file for the resonators
design_data_example = OperateTuneData(design_file_path=design_file)
# get the UFM layout metadata (mux_layout_position and bond_pad mapping)
layout_data_example = get_layout_data(waferfile, dark_bias_lines=dark_bias_lines, plot=False)


In [3]:
# read the tunefile and initialize the data instance
tune_data_smurf = make_map_smurf(tunefile=tunefile, north_is_highband=north_is_highband,
                                 design_data=design_data_example,
                                 layout_position_path=mux_pos_num_to_mux_band_num_path,
                                 layout_data=layout_data_example,
                                 csv_filename=output_filename)


In [4]:
# VNA scans
tune_data_vna = make_map_vna(tune_data_vna_output_filename=tune_data_vna_output_filename,
                             path_north_side_vna=path_north_side_vna,
                             path_south_side_vna=path_south_side_vna,
                             north_is_highband=north_is_highband, shift_mhz=vna_shift_mhz,
                             design_data=design_data_example,
                             layout_position_path=mux_pos_num_to_mux_band_num_path,
                             layout_data=layout_data_example,
                             csv_filename=output_filename_vna)


In [5]:
# Optical power data from validation of dark bias lines.
_cold_ramp_data_by_column, cold_ramp_data_by_row = read_csv(path=cold_ramp_file)
coldload_ivs = [data_row for data_row in cold_ramp_data_by_row if data_row['note'].lower() == 'iv']


In [6]:
#read in the sample psat data.
_psat_by_band_chan, psat_by_temp = read_psat(coldload_ivs=coldload_ivs, make_plot=False)


In [7]:
# Plot variables
temp_k = 9.0
show_plot = False
save_plot = True

# example plots
for bandpass_target, psat_min, psat_max in [(90, 0.0, 3.0e-12),
                                            (150, 0.0, 6.0e-12)]:
    tune_data_smurf.plot_with_psat(psat_by_temp=psat_by_temp, bandpass_target=bandpass_target,
                                   temp_k=temp_k, psat_min=psat_min, psat_max=psat_max,
                                   show_plot=show_plot, save_plot=save_plot)

Plot saved at: plots\90_GHz_Psat_at_100mK_CL=9.0K,_range=0.0-3.0_pW.png
Plot saved at: plots\150_GHz_Psat_at_100mK_CL=9.0K,_range=0.0-6.0_pW.png
